In [64]:
import numpy as np
import pyvista as pv
# import voronoi
from scipy.spatial import Voronoi, voronoi_plot_2d

pv.global_theme.allow_empty_mesh = True

In [60]:
def to_pv_style(array_2d, illegal):
    style = []
    for a in array_2d:

        if -1 in a:
            continue
        
        a_clean = [a_i for a_i in a if a_i not in illegal]
        la = len(a_clean)
        if la == 0:
            continue
        style.append(la)
        for aa in a_clean:
            style.append(aa)

    return style

In [78]:
points = np.load('poss_erda.npy')

# Step 2: Compute the Voronoi diagram
vor = Voronoi(points)

vor_points = vor.vertices

vor_faces = vor.regions


# remove points that are too far away
illegal = []
for i, p in enumerate(vor_points):
    if np.linalg.norm(p) > 60:
        illegal.append(i)

vor_faces = to_pv_style(vor_faces, illegal)


# Step 3: Create a PyVista mesh of the Voronoi diagram
vor_mesh = pv.PolyData()
vor_mesh.points = vor_points
vor_mesh.faces = np.array(vor_faces)


# Step 4: Plot the Voronoi diagram
p = pv.Plotter()
p.add_mesh(vor_mesh,)
# p.add_points(points, color='r')
p.show()

Widget(value='<iframe src="http://localhost:51005/index.html?ui=P_0x1d2b6a2d8b0_16&reconnect=auto" class="pyvi…

Not exactly looking perfect :(

![alt text](viz_screenshot.png)